In [1]:
import re
from ast import parse
from datetime import datetime
import yfinance as yf
import pandas as pd

In [9]:
def parse_option_ticker(ticker_str):
    m=re.match(r"^([A-Z]+)(\d{6})([CP])(\d+(\.\d+)?)$",ticker_str)
    if not m:
        raise ValueError(f"Unrecognized ticker format: {ticker_str}")

    underlying=m.group(1)
    expiry_str=m.group(2)
    option_type="Call" if m.group(3)=="C" else "Put"
    strike=float(m.group(4))

    expiry_date= datetime.strptime(expiry_str,"%y%m%d").date()
    return{
        "Underlying": underlying,
        "Expiry Date": expiry_date,
        "Strike": strike,
        "Type": option_type

    }

In [10]:
def classify_moneyness(spot,strike,opt_type):
    tol=0.005*spot
    if abs(spot-strike)<=tol:
        return "ATM"
    if opt_type=="Call":
        return "ITM" if spot>strike else"OTM"
    else:
        return "OTM" if spot>strike else"ITM"

In [14]:
opt_ticker="AAPL240621C125"
parsed=parse_option_ticker(opt_ticker)
spot=yf.Ticker(parsed["Underlying"]).history(period="1d")["Close"].iloc[-1]
moneyness=classify_moneyness(spot,parsed["Strike"],parsed["Type"])
parsed["spot"]=round(spot,2)
parsed["Moneyness"]=moneyness
print(parsed)

{'Underlying': 'AAPL', 'Expiry Date': datetime.date(2024, 6, 21), 'Strike': 125.0, 'Type': 'Call', 'spot': np.float64(254.63), 'Moneyness': 'ITM'}


In [15]:
tickers = ["AAPL240621C125", "AAPL240621P120", "AAPL240621C150"]
rows = []
for t in tickers:
    p = parse_option_ticker(t)
    spot = yf.Ticker(p["Underlying"]).history(period="1d")["Close"].iloc[-1]
    p["Spot"] = round(spot, 2)
    p["Moneyness"] = classify_moneyness(spot, p["Strike"], p["Type"])
    rows.append(p)

df = pd.DataFrame(rows)
print("\nMultiple option chain classification:")
print(df)


Multiple option chain classification:
  Underlying Expiry Date  Strike  Type    Spot Moneyness
0       AAPL  2024-06-21   125.0  Call  254.63       ITM
1       AAPL  2024-06-21   120.0   Put  254.63       OTM
2       AAPL  2024-06-21   150.0  Call  254.63       ITM
